# Análisis Exploratorio y Limpieza de Datos (Refactorizado para Ejecución Local)

## 1. Configuración de Entorno y Carga de Datos

Se importan las librerías necesarias, se configura el entorno y se cargan los datos crudos desde el directorio local `data/`.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import sys
import os

# Add src directory to path to import our custom module
# This makes the notebook runnable from the 'notebooks' directory
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))
from data_cleaner import run_cleaning_pipeline

# Set plotting style
sns.set_theme(style="whitegrid")

# Define relative file paths
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
input_file_path = os.path.join(project_root, 'data', 'rentabilidad_productos.csv')
output_file_path = os.path.join(project_root, 'data', 'rentabilidad_productos_limpio.parquet')

In [ ]:
# Load the raw dataset
try:
    df_raw = pd.read_csv(input_file_path)
    print(f"DataFrame crudo cargado con {df_raw.shape[0]} filas y {df_raw.shape[1]} columnas.")
    display(df_raw.head())
except FileNotFoundError:
    print(f"Error: El archivo no se encontró en la ruta esperada: {input_file_path}")
    print("Por favor, asegúrese de que 'rentabilidad_productos.csv' exista en el directorio 'data'.")

## 2. Análisis Exploratorio de Datos Crudos (EDA)

Se explora el DataFrame original para identificar problemas de calidad de datos antes de la limpieza.

### 2.1. Tipos de Datos y Resumen General

In [ ]:
df_raw.info()

### 2.2. Análisis de Valores Nulos

In [ ]:
missing_values = df_raw.isnull().sum()
missing_percentage = (missing_values / len(df_raw)) * 100
missing_df = pd.DataFrame({'count': missing_values, 'percentage': missing_percentage})
missing_df = missing_df[missing_df['count'] > 0].sort_values(by='count', ascending=False)

print(f"Se encontraron {len(missing_df)} columnas con valores nulos en los datos crudos.")
display(missing_df)

msno.matrix(df_raw)
plt.show()

### 2.3. Análisis de Duplicados en Clave Primaria (SKU)

In [ ]:
# We need to normalize column names first to access 'sku' reliably
temp_df = df_raw.copy()
temp_df.columns = temp_df.columns.str.strip().str.lower()

sku_counts = temp_df.sku.value_counts()
duplicated_skus = sku_counts[sku_counts > 1]

print(f"Análisis de duplicados de SKU:")
print(f"- Hay {len(sku_counts)} SKUs únicos de un total de {len(temp_df)} registros.")
print(f"- Se encontraron {len(duplicated_skus)} SKUs con registros duplicados.")

## 3. Proceso de Limpieza de Datos

Se invoca el pipeline de limpieza definido en el módulo `data_cleaner` para procesar el DataFrame.

In [ ]:
df_cleaned = run_cleaning_pipeline(df_raw)

## 4. Verificación y Análisis de Datos Limpios

Se realizan una serie de verificaciones y visualizaciones sobre el DataFrame limpio para asegurar la calidad del proceso.

### 4.1. Verificación de Tipos de Datos y Nulos

In [ ]:
print("--- Tipos de datos después de la limpieza ---")
df_cleaned.info()

print("\n--- Conteo de Nulos Restantes ---")
print(df_cleaned.isnull().sum())

### 4.2. Verificación de Integridad de Datos (Cálculo de Margen)

In [ ]:
df_cleaned['margen_calculado'] = df_cleaned['precio_venta'] - df_cleaned['precio_compra']
df_cleaned['margen_diferencia'] = (df_cleaned['margen'] - df_cleaned['margen_calculado']).abs()

discrepancies = df_cleaned[df_cleaned['margen_diferencia'] > 0.01] # Use tolerance for float precision

print(f"Se encontraron {len(discrepancies)} filas con una diferencia mayor a 0.01 entre el margen reportado y el calculado.")

### 4.3. Resumen Estadístico del DataFrame Limpio

In [ ]:
display(df_cleaned.describe(include='all').T)

### 4.4. Distribución de Variables Numéricas Limpias

In [ ]:
for col in ['precio_compra', 'precio_venta', 'margen']:
    fig, axes = plt.subplots(1, 2, figsize=(15, 4))
    fig.suptitle(f"Análisis de la columna numérica limpia: '{col}'", fontsize=16)

    sns.histplot(df_cleaned[col], kde=True, ax=axes[0])
    axes[0].set_title('Distribución (Histograma)')

    sns.boxplot(x=df_cleaned[col], ax=axes[1])
    axes[1].set_title('Diagrama de Caja (Boxplot)')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

## 5. Almacenamiento del DataFrame Limpio

Se guarda el DataFrame limpio y validado en formato Parquet en el directorio `data/`.

In [ ]:
# Drop temporary columns used for verification before saving
final_df_to_save = df_cleaned.drop(columns=['margen_calculado', 'margen_diferencia'])

final_df_to_save.to_parquet(output_file_path, index=False, engine='pyarrow')

print(f"DataFrame limpio con {len(final_df_to_save)} filas guardado exitosamente en: {output_file_path}")